In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2
from glob import glob
import tensorflow as tf

# from src.data.dataloader import DataGenerator
from src.data.augmentation import random_crop, random_rotate, random_flip_left_right
from src.data.dataloader import build_tf_dataloader, parser_DIODE_dataset
from src.helpers import setup_gpu
from src.viz.plot_images import visualize_depth_map
setup_gpu(0)

# Data preparation

In [ ]:
validation_df = pd.read_csv('/workspace/tmp/DIODO_val.csv')
validation_df.head(2)

In [ ]:
transform = [ parser_DIODE_dataset, random_crop, random_rotate]
dataloader = build_tf_dataloader( validation_df['image'].values,
                                  validation_df['depth'].values,
                                  validation_df['mask'].values,
                                  batch_size=16,
                                  transforms=transform,
                                  train=True,
                                  )

sample = next(iter( dataloader ))
_ = visualize_depth_map(sample)
plt.show()

In [ ]:
# idx = np.random.randint(0, len(validation_df))
# mask = np.load(validation_df['mask'].values[idx])
# mask = np.expand_dims(mask, axis=-1)

# depth = np.load(validation_df.depth.values[idx])
# depth = np.clip(depth, 0.6, 350) / 350.0

# print(depth.shape)
# print(depth.min(), depth.mean(), depth.max())

# # max_depth = min(300, depths.max())
# # depths = np.clip(depths, 0.1, max_depth)
# # depths = np.log(depths)
# # depths = np.ma.masked_where(~(masks > 0), depths)
# # depths = np.clip(depths, 0.1, np.log(max_depth))

# viz_depth = np.log(depth)
# viz_depth = np.ma.masked_where( ~(mask>0), viz_depth )


# cmap = plt.cm.get_cmap("jet").copy()
# cmap.set_bad(color="black")
# plt.imshow( viz_depth , cmap=cmap)
# plt.show()

# # depth in m
# # –  Return Density: 99.6% (indoor) / 66.9% (outdoor)
# # –  Depth Precision: ±1 mm
# # –  Angular Resolution: 0.009°
# # –  Max Range: 350 m
# # –  Min Range: 0.6 m

# Dataloader

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def read_image(image_path):
    content = tf.io.read_file(image_path)
    ext = tf.strings.split(image_path, '.')[-1]
    ext = tf.strings.lower(ext)

    bmp_decode  = lambda: tf.image.decode_bmp(content, channels=3)
    png_decode  = lambda: tf.image.decode_png(content, channels=3)
    jpeg_decode = lambda: tf.image.decode_jpeg(content, channels=3, try_recover_truncated=True)

    image = tf.case([ (ext == tf.constant('bmp'), bmp_decode),
                    (ext == tf.constant('png'), png_decode),
                    (ext == tf.constant('jpg'), jpeg_decode),
                    (ext == tf.constant('jpeg'), jpeg_decode),
                  ], default=bmp_decode)
    image = tf.image.convert_image_dtype(image, dtype='float32') #will be in [0, 1]
    return image

def load_images(filenameA, filenameB):
    return read_image(filenameA), read_image(filenameB)

def load_numpy(filename):
    filename = filename.numpy().decode('utf-8')
    np_loaded = np.load(filename)
    return np_loaded

def load_image_depth_mask(input_path, depth_path, mask_path):
    # read as RGB and convert to [0, 1], float32
    input_image = read_image(input_path)
    h, w = input_image.shape[:2]
    # load depth map
    [depth,] = tf.py_function(load_numpy, [depth_path], [tf.float32])
    depth.set_shape( [h,w,1] )
    # load valid mask
    [mask,] = tf.py_function(load_numpy, [mask_path], [tf.float32])
    # put mask in [H,W,1] format
    mask = tf.expand_dims(mask, axis=-1)
    mask.set_shape( [h,w,1] )
    return input_image, depth, mask

def basic_image_depth_mask_preproc(input_image, depth, mask, min_max_depth=[0.1, 350.0]):

    # depth = tf.clip_by_value(depth, min_max_depth[0], min_max_depth[1])
    # depth = tf.math.log(depth)
    # depth = depth * (1.0 - mask)
    # depth = tf.clip_by_value(depth, min_max_depth[0], tf.math.log(min_max_depth[1]))

    return input_image, depth, mask

def build_tf_dataloader(input_paths, depth_paths, mask_paths, batch_size=32, transforms=[], train=True):
    data = tf.data.Dataset.from_tensor_slices( (input_paths, depth_paths, mask_paths) )
    if train:
        data = data.shuffle(1024)

    data = data.map(load_image_depth_mask, num_parallel_calls=tf.data.AUTOTUNE)
    # data = data.map(basic_image_depth_mask_preproc, num_parallel_calls=tf.data.AUTOTUNE)

    for transform_f in transforms:
        data = data.map(transform_f, num_parallel_calls=tf.data.AUTOTUNE)
    # data = data.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    data = data.batch(batch_size)
    return data

In [ ]:
ib, db, mb = load_image_depth_mask(validation_df['image'].values[0],
                                   validation_df['depth'].values[0],
                                   validation_df['mask'].values[0],
                                   )

ib, db, mb = basic_image_depth_mask_preproc(ib, db, mb)

In [ ]:
dataset = build_tf_dataloader( validation_df['image'].values,
                               validation_df['depth'].values,
                               validation_df['mask'].values,
                               batch_size=25,
                               transforms=[],
                               train=True,
                            )

In [ ]:
len(dataset)

In [ ]:
images, depths, masks = next(iter(dataset))

images = images.numpy()
depths = depths.numpy()
masks = masks.numpy()

In [ ]:
depths.min(), depths.max()

In [ ]:
depths.min(), depths.max()

In [ ]:
# max_depth = min(300, np.percentile(depths, 99))
max_depth = min(300, depths.max())
depths = np.clip(depths, 0.1, 350)
depths = np.log(depths)
depths = np.ma.masked_where(~(masks > 0), depths)
depths = np.clip(depths, 0.1, np.log(max_depth))

cmap = plt.cm.get_cmap("jet").copy()
cmap.set_bad(color="black")

fig, ax = plt.subplots(6, 2, figsize=(50, 50))
for i in range(6):
    ax[i, 0].imshow((images[i].squeeze()))
    ax[i, 1].imshow((depths[i].squeeze()), cmap=cmap)
plt.show()